In [1]:
import pandas as pd

In [32]:
def feature_engineering(df_train):
    df_train['first_timestamp'] = pd.to_datetime(df_train['first_timestamp'])

    # day
    df_train['Day'] = df_train['first_timestamp'].dt.day
    # month
    df_train['Month'] = df_train['first_timestamp'].dt.month
    # hour
    df_train['Start_hour'] = df_train['first_timestamp'].dt.hour
    # minute
    df_train['Start_minute'] = df_train['first_timestamp'].dt.minute
    # second
    df_train['Start_second'] = df_train['first_timestamp'].dt.second
    # Monday is 0 and Sunday is 6
    df_train['Start_weekday'] = df_train['first_timestamp'].dt.weekday
    # week of the year
    df_train['Start_week_of_year'] = df_train['first_timestamp'].dt.week

    df_train.drop('first_timestamp', inplace=True, axis=1)

    return df_train

In [33]:
df_train = pd.read_csv('../../data/prefix_data/full_prefix_train.csv', index_col=0).fillna(0)
df_test = pd.read_csv('../../data/prefix_data/full_prefix_test.csv', index_col=0).fillna(0)

df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)


X_train = df_train.drop(columns=['case_outcome'], inplace=False)
y_train = df_train['case_outcome']

X_test = df_test.drop(columns=['case_outcome'], inplace=False)
y_test = df_test['case_outcome']

/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_23342/1275838501.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_train['Start_week_of_year'] = df_train['first_timestamp'].dt.week
/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_23342/1275838501.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_train['Start_week_of_year'] = df_train['first_timestamp'].dt.week


In [36]:
col_dict = {}
dtypes_list = dict(df_train.dtypes)
for pair in dtypes_list.items():
    key = str(pair[1])
    col_dict[key] = col_dict.get(key, []) + [pair[0]]

col_dict["object"].pop()
col_dict['object']

['Accepted', 'Selected', 'case:LoanGoal', 'case:ApplicationType']

In [37]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [38]:
categorical_pipe = make_pipeline(OneHotEncoder())

# Using categorical pipe for features A,B,C, numeric pipe otherwise
preprocessor = make_column_transformer((categorical_pipe, col_dict['object']), remainder='passthrough')

# Combine with learning algorithm in another pipeline

pipe = make_pipeline(preprocessor, RandomForestClassifier())

In [39]:
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

accuracy_score(y_pred=y_pred, y_true=y_test)

0.8481047989430789